In [2]:
import requests
import pattern.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from progressbar import ProgressBar
from bs4 import BeautifulSoup

<br><br/> 

In [3]:
class build_dataset(object):
    
    def __init__(self, terms=None):
        
        self.names_ = []
        self.genres_ = [] 
        self.genreIDs_ = []
        self.IDs_desc_ = []
        self.terms = terms 
        
        
        


    def pull_genres(self, my_data):
        

        all_genres = [] 
        all_genreIDs = []
        all_genreswithIDs = [] 
        new_list_IDs = []
        urls = [] 
        rss_feeds = [] 
        names = []
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL'] 

        
        pbar = ProgressBar()
        
        
        
        
        if my_data["resultCount"] != 0: 
            for elem in pbar(my_data["results"]):   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                        
                        if "trackCount" in elem.keys():
                            if elem["trackCount"] >= 20: 
                                
                                episode_count = elem["trackCount"]
                                
                                if "genreIds" in elem.keys():
                                    genreID = elem["genreIds"]
                                    all_genreIDs.append(genreID)

                                if "genres" in elem.keys():
                                    genre = elem["genres"] 
                                    all_genres.append(genre)
                                    genresandIDs = list(zip(genreID, genre))
                                    all_genreswithIDs.append(genresandIDs)

                                if "collectionName" in elem.keys():
                                    collection_name = elem["collectionName"]
                                    names.append(collection_name)


                                if "collectionViewUrl" in elem.keys(): 
                                    itunes_url = elem["collectionViewUrl"]
                                                                                           
                                elif "trackViewUrl" in elem.keys():
                                    itunes_url = elem["trackViewUrl"]
                                    

                                if "feedUrl" in elem.keys():
                                    feedurl = elem["feedUrl"] 
                                    
                                    
                                if "artworkUrl100" in elem.keys():
                                    artwork = elem["artworkUrl100"]
                                    
                                


                                my_row = [collection_name, artwork, episode_count, genreID, itunes_url, feedurl]
                                my_row_pd = pd.DataFrame([my_row], columns = columns)                    
                                self.df_podcast_ = self.df_podcast_.append(my_row_pd, ignore_index=True)



                                self.genres_ = list(set(np.concatenate(all_genres).tolist()))
                                self.genreIDs_ = list(set(np.concatenate(all_genreIDs).tolist()))
                                self.names_ = names  
                                new_list_IDs = np.concatenate(all_genreswithIDs).tolist()
                                self.IDs_desc_ = [ii for n,ii in enumerate(new_list_IDs) if ii not in new_list_IDs[:n]]
                        
                        else:
                            pass 


                    else:
                        pass 

                else:
                    pass 
                   
        else: 
            pass 
        
        return self
        

    
    def build_random_data(self, term):
        
        
        url = "https://itunes.apple.com/search?term=" + term + "&limit=200&country=US&lan=en_us&entity=podcast"
        
        try:
            response = requests.get(url) 
            data = response.json()

            print (data["resultCount"])
            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 



    def build_genres_data(self, genre):
        
        url = "https://itunes.apple.com/search?term=" + genre + "&limit=200&country=US&lan=en_us&entity=podcast"
        
        try: 
            response = requests.get(url) 
            data = response.json()

            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 


        
        
    def find_genres(self, gnr):
        
        self.build_genres_data(gnr)

        genres_list = self.genres_
        names_list = self.names_
        IDs_genres_list = self.IDs_desc_

        if names_list != []: 
            for name in names_list:
                if name not in self.initial_names_:
                    self.initial_names_.append(name)
        
        if IDs_genres_list != []:
            for item in IDs_genres_list:
                if item not in self.genres_with_IDs_:
                    self.genres_with_IDs_.append(item)

        if genres_list != []: 
            for genre in genres_list:
                if genre not in self.initial_genres_:
                    self.initial_genres_.append(genre)
                    print ("extra")
                    self.find_genres(genre)
                  
        return self 
                    
        

    def initialize(self):
        self.initial_genres_ = None
        self.genreIDs_ = None
        self.names_ = None 
        self.genres_with_IDs_ = None
        
        url = "https://itunes.apple.com/search?term=podcasts&limit=200&country=US&lan=en_us&entity=podcast" 
        
        
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL']
        self.df_podcast_ = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns = columns)

        try: 
            response = requests.get(url) 
            my_data = response.json()

            if my_data["resultCount"] != 0:
                self.pull_genres(my_data)

                self.initial_genres_ = self.genres_
                self.initial_genreIDs_ = self.genreIDs_
                self.genres_with_IDs_ = self.IDs_desc_
                self.initial_names_ = self.names_
                

            else: 
                pass 
            
        except ValueError:
            print ("Decoding JSON has failed")
        

                    
    def build_data(self):
        
        terms = self.terms
        self.initialize()
        
        if self.initial_genres_ is not None:
            for initial_genre in self.initial_genres_: 
                self.find_genres(initial_genre)
        
        print (terms)
        if terms is not None:
            for term in terms:
                self.build_random_data(term)
        

        return self 
        

In [4]:
more_genres = ["random", "editorial", "review", "hobbies", "fiction", "mystery", "adventure", "romance", "humor"]

In [5]:
data_builder = build_dataset(more_genres)

In [6]:
data_builder.build_data()

N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  3% (6 of 200) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00
  5% (11 of 200) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:02
  9% (18 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:02
 11% (23 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:02
 15% (31 of 200) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:02
 19% (38 of 200) |####                   | Elapsed Time: 0:00:00 ETA:   0:00:01
 22% (44 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:01
 25% (51 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:01
 27% (54 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:01
 28% (57 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:01
 29% (59 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:01
 30% (61 of 200) |#######               

extra


N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (3 of 200) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00
  4% (8 of 200) |                        | Elapsed Time: 0:00:00 ETA:   0:00:02
  5% (11 of 200) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:02
  9% (18 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:02
 13% (26 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:02
 18% (36 of 200) |####                   | Elapsed Time: 0:00:00 ETA:   0:00:01
 22% (44 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:01
 25% (51 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:01
 29% (59 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:01
 33% (66 of 200) |#######                | Elapsed Time: 0:00:00 ETA:   0:00:01
 37% (74 of 200) |########               | Elapsed Time: 0:00:00 ETA:   0:00:01
 41% (82 of 200) |#########             

extra


N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  5% (11 of 200) |#                      | Elapsed Time: 0:00:00 ETA:  00:00:00
 11% (23 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:00
 16% (33 of 200) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:00
 22% (44 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:00
 25% (51 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:00
 29% (59 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:00
 34% (69 of 200) |#######                | Elapsed Time: 0:00:00 ETA:   0:00:00
 38% (76 of 200) |########               | Elapsed Time: 0:00:00 ETA:   0:00:00
 42% (84 of 200) |#########              | Elapsed Time: 0:00:00 ETA:   0:00:00
 46% (92 of 200) |##########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 49% (99 of 200) |###########            | Elapsed Time: 0:00:00 ETA:   0:00:00
 54% (109 of 200) |###########          

...

extra


N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  2% (5 of 200) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04
  4% (8 of 200) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04
  6% (13 of 200) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:03
  9% (18 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:03
 11% (23 of 200) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:02
 14% (28 of 200) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:02
 15% (30 of 200) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:03
 20% (41 of 200) |####                   | Elapsed Time: 0:00:00 ETA:   0:00:02
 24% (49 of 200) |#####                  | Elapsed Time: 0:00:00 ETA:   0:00:02
 28% (56 of 200) |######                 | Elapsed Time: 0:00:00 ETA:   0:00:01
 34% (69 of 200) |#######                | Elapsed Time: 0:00:00 ETA:   0:00:01
 41% (82 of 200) |#########             

extra
extra


N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


UnboundLocalError: local variable 'feedurl' referenced before assignment

In [9]:
genre_IDs = data_builder.genres_with_IDs_

with open('genre_IDs.txt','a') as t:
    t.write('\n' + str(genre_IDs))      
    t.close()

In [10]:
podcast_dataframe = data_builder.df_podcast_.drop_duplicates(['Name']).reset_index(drop=True)

In [11]:
podcast_dataframe.drop(podcast_dataframe.index[0], inplace=True)

In [12]:
podcast_dataframe = podcast_dataframe.reset_index(drop=True)

In [13]:
podcast_dataframe.to_csv("df_01.csv", index=False)

In [16]:
podcast_dataframe.head()

,Name,Artwork,Episode Count,GenreIDs,iTunes URL,FeedURL
0,The Joe Rogan Experience,https://is5-ssl.mzstatic.com/image/thumb/Podca...,300,"[1303, 26, 1324]",https://podcasts.apple.com/us/podcast/the-joe-...,http://joeroganexp.joerogan.libsynpro.com/rss
1,Stuff You Should Know,https://is5-ssl.mzstatic.com/image/thumb/Podca...,300,"[1324, 26]",https://podcasts.apple.com/us/podcast/stuff-yo...,https://feeds.megaphone.fm/stuffyoushouldknow
2,Serial,https://is4-ssl.mzstatic.com/image/thumb/Podca...,36,"[1489, 26, 1488]",https://podcasts.apple.com/us/podcast/serial/i...,http://feeds.serialpodcast.org/serialpodcast
3,Real Time with Bill Maher,https://is2-ssl.mzstatic.com/image/thumb/Podca...,300,"[1489, 26]",https://podcasts.apple.com/us/podcast/real-tim...,http://billmaher.hbo.libsynpro.com/rss
4,Fresh Air,https://is3-ssl.mzstatic.com/image/thumb/Podca...,50,"[1482, 26, 1301, 1309]",https://podcasts.apple.com/us/podcast/fresh-ai...,https://www.npr.org/rss/podcast.php?id=381444908
